# Data Cleaning and Modeling
This notebook is to import, clean, and save data from the specified paths in config.ini


These variables are for transforming the challenge's *.csv* files.
* *to_drop*
<br>A list of columns to drop from the data.
* *dtype_dict*
<br>A dictionary containing the specified dtypes for columns.
* *fillna_dict*
<br>A dictionary containing fill values for missing data. This is for the cleaning
process.
---

Import needed libraries and created functions

In [38]:
import sys 
sys.path.append('../')
# Imports needed libraries
# and created functions
from src.etl_functions import *

Import configuration

In [39]:
config = configparser.ConfigParser()
config.read("src/config.ini")

['src/config.ini']

In [40]:
# Specified output path
output = config['paths']['data_path']
# Specified data paths
trn_data = config['paths']['train_data']
trn_lbls = pd.read_csv(config['paths']['train_labels'])
tst_data = config['paths']['test_data']
# Import submission format
sub_form = pd.read_csv(config['paths']['sub_form'])

In [41]:
to_drop = ['extraction_type', 'extraction_type_group',
            'management_group',
            'payment_type',
            'quantity_group',
            'source_type','source_class', 
            'waterpoint_type_group',
            'district_code', 
            'construction_year',
            'num_private',
            'recorded_by',
            'id',
            'scheme_name', 
            'date_recorded']

fill_dict = {'funder':'Other',
                'installer': 'Other',
                'subvillage': 'Other', 
                'public_meeting': False,
                'scheme_management': 'Unknown',
                'permit': False}

dtype_dict = {'amount_tsh': 'float32',
            'funder': 'category',
            'gps_height': 'int16',
            'installer': 'category',
            'longitude': 'float16',
            'latitude': 'float16',
            'wpt_name': 'category',
            'num_private': 'int16',
            'basin': 'category',
            'subvillage': 'category',
            'region': 'category',
            'region_code': 'int8',
            'district_code': 'int8',
            'lga': 'category',
            'ward': 'category',
            'population': 'int16',
            'recorded_by': 'category',
            'scheme_management': 'category',
            'construction_year': 'int16',
            'extraction_type': 'category',
            'extraction_type_group': 'category',
            'extraction_type_class': 'category',
            'management': 'category',
            'management_group': 'category',
            'payment': 'category',
            'payment_type': 'category',
            'water_quality': 'category',
            'quality_group': 'category',
            'quantity': 'category',
            'quantity_group': 'category',
            'source': 'category',
            'source_type': 'category',
            'source_class': 'category',
            'waterpoint_type': 'category',
            'waterpoint_type_group': 'category'}

In [42]:
train_df, test_df, exp_output = get_cleaned_sets(trn_data,tst_data, to_drop, output, fill_dict,
                                                dtype_dict=dtype_dict,
                                                return_output=True)

C:\Users\Cristian\AppData\Roaming\Python\Python310\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\Cristian\AppData\Roaming\Python\Python310\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Cleaning successful.
Associated time is 280722_0947PM


In [43]:
trn_lbls.index=trn_lbls['id']
trn_lbls.drop(columns='id', inplace=True)


In [44]:
rf = RandomForestClassifier(n_estimators=150, random_state=42)
rf.fit(train_df, trn_lbls)
preds = rf.predict(test_df)
sub_form['status_group'] = preds
sub_form.to_csv(f'{exp_output}submission.csv',index=False)
with open(f"{exp_output}experiment_notes.txt", 'a') as f:
        f.write(str(rf.set_params()))

C:\Users\Cristian\AppData\Local\Temp\ipykernel_20472\2451278971.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(train_df, trn_lbls)


In [45]:
sub_form

,id,status_group
0,50785,non functional
1,51630,functional
2,17168,functional
3,45559,non functional
4,49871,functional
...,...,...
14845,39307,non functional
14846,18990,functional
14847,28749,functional
14848,33492,functional


In [46]:
from sklearn.model_selection import GridSearchCV
rf = RandomForestClassifier()
param_grid = {'n_estimators':[50,100,150,200],
              'criterion': ['gini', 'entropy', 'log_loss'],
              'max_features': ['sqrt', 'log2'],
              'random_state':[42]
              }
gs = GridSearchCV(rf,param_grid=param_grid)
gs.fit(train_df, trn_lbls)
preds = gs.predict(test_df)
sub_form['status_group'] = preds
sub_form['status_group'] = preds
sub_form.to_csv(f'{exp_output}submission.csv',index=False)
gs.best_estimator_

C:\Users\Cristian\AppData\Roaming\Python\Python310\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Cristian\AppData\Roaming\Python\Python310\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Cristian\AppData\Roaming\Python\Python310\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Cristian\AppData\Roaming\Python\Python310\site-packages